In [1]:
import sys,os
import tarfile
import numpy as np
import pandas as pd
import xarray as xr
import time as tm

import SDFC.link as sdl
import NSSEA as ns
import NSSEA.plot as nsp
import NSSEA.models as nsm

In [2]:
from cmdstanpy import CmdStanModel
import logging
from cmdstanpy import cmdstan_path, set_cmdstan_path

set_cmdstan_path('/home/barbauxo/miniconda3/envs/TestSDFC/bin/cmdstan')
cmdstan_path()

'/home/barbauxo/miniconda3/envs/TestSDFC/bin/cmdstan'

In [3]:
#Useful modified
def correct_miss( X , lo =  100 , up = 350 ):##{{{
#	return X
	mod = str(X.columns[0])
	bad = np.logical_or( X < lo , X > up )
	bad = np.logical_or( bad , np.isnan(X) )
	bad = np.logical_or( bad , np.logical_not(np.isfinite(X)) )
	if np.any(bad):
		idx,_ = np.where(bad)
		idx_co = np.copy(idx)
		for i in range(idx.size):
			j = 0
			while idx[i] + j in idx:
				j += 1
			idx_co[i] += j
		X.iloc[idx] = X.iloc[idx_co].values
	return X
##}}}

def load_models_CMIP6(pathInp,type_data):
    ## List of models X
    pathInpX= os.path.join(pathInp,"CMIP6/X",type_data)
    modelsX = [  "_".join(f.split("/")[-1][:-3].split("_")[-2:]) for f in os.listdir(pathInpX) ]
    modelsX.sort()

    ## List of models Y
    pathInpY= os.path.join(pathInp,"CMIP6/Y",type_data)
    modelsY = [ "_".join(f.split("/")[-1][:-3].split("_")[-2:]) for f in os.listdir(pathInpY) ]
    modelsY.sort()
    models = list(set(modelsX) & set(modelsY))
    models.sort()
    print(models)


    ## Load X and Y
    lX = []
    lY = []
    if type_data== "03_Post_treatment":
    	
    	for m in models:
		
        	## Load X
        
        	df   = xr.open_dataset( os.path.join( pathInpX , "full_Europe_tas_YearMean_ssp585_{}.nc".format(m) ) ,decode_times=True )
        	time = df.time["time.year"].values.astype(int)
        	X    = pd.DataFrame( df.tas.values.ravel() , columns = [m] , index = time )
        	
        	lX.append( correct_miss(X , lo =  -15 , up = 25))
    
        	## Load Y
        	df   = xr.open_dataset( os.path.join( pathInpY , "full_Tricastin_ssp585_{}.nc".format(m) ) ,decode_times=True  )
        	time = df.time["time.year"].values.astype(int)
        	Y    = pd.DataFrame( df.tasmax.values.ravel() , columns = [m] , 	index = time )
        	lY.append( correct_miss(Y, lo =  -15 , up = 25 ))
    else:
    	for m in models:

        	## Load X
        
        	df   = xr.open_dataset( os.path.join( pathInpX , "full_Europe_tas_YearMean_ssp585_{}.nc".format(m) ) ,decode_times=False )
        	time = df.time.values.astype(int)
        	X    = pd.DataFrame( df.tas.values.ravel() , columns = [m] , index = time )
    
        	lX.append( correct_miss(X) )
    
        	## Load Y
        	df   = xr.open_dataset( os.path.join( pathInpY , "full_Tricastin_ssp585_{}.nc".format(m) ) ,decode_times=False  )
        	time = df.time.values.astype(int)
        	Y    = pd.DataFrame( df.tasmax.values.ravel() , columns = [m] , 	index = time )
        	lY.append( correct_miss(Y) )
    return lX, lY,models
    

def load_obs(pathInp,type_data):
    ## Load Observations
    dXo = xr.open_dataset(os.path.join( pathInp ,"Observations",type_data,"Xo.nc"))
    Xo  = pd.DataFrame( dXo.tas.values.squeeze() , columns = ["Xo"] , index = dXo.time["time.year"].values )

    Xo #Deja en anomalies
    dYo = xr.open_dataset(os.path.join( pathInp ,"Observations",type_data,"Yo.nc"))
    Yo  = pd.DataFrame( dYo.TX.values.squeeze() , columns = ["Yo"] , index = dYo.time["time.year"].values )
    return Xo,Yo #en celsius

In [4]:
#Parameters
time_period    = np.arange( 1850 , 2101 , 1 , dtype = int )
time_reference = np.arange( 1986 , 2016 , 1 , dtype = int )

ci          = 0.05
sample_dis=False #If want n sample of each GCM model. For graph only, not used for multisynthesis. Takes a lot of time

#Time period of interest
T=100
T1=2000
T2=2100
deb=1850
fin=2101

ns_law      = nsm.GEV()
event       = ns.Event( "HW19" , 2019 , time_reference , type_ = "value" , variable = "TX3X" , unit = "K" )
verbose     = "--not-verbose" not in sys.argv

In [5]:
pathInp='/home/barbauxo/Documents/Doctorat/03_Travail/2023_01 Application Tricastin/Data'
type_data="03_Post_treatment"  #"02_Selected"
lX,lY,models=load_models_CMIP6(pathInp,type_data)
Xo,Yo=load_obs(pathInp,type_data)
event.value = float(Yo.loc[event.time])

['ACCESS-CM2_i1p1f1', 'ACCESS-ESM1-5_i1p1f1', 'CMCC-ESM2_i1p1f1', 'CNRM-CM6-1-HR_i1p1f2', 'CNRM-CM6-1_i1p1f2', 'CNRM-ESM2-1_i1p1f2', 'CanESM5_i1p2f1', 'EC-Earth3-CC_i1p1f1', 'EC-Earth3-Veg-LR_i1p1f1', 'EC-Earth3-Veg_i1p1f1', 'EC-Earth3_i1p1f1', 'FGOALS-g3_i1p1f1', 'GFDL-ESM4_i1p1f1', 'HadGEM3-GC31-LL_i1p1f3', 'HadGEM3-GC31-MM_i1p1f3', 'INM-CM4-8_i1p1f1', 'INM-CM5-0_i1p1f1', 'IPSL-CM6A-LR_i1p1f1', 'KACE-1-0-G_i1p1f1', 'MIROC-ES2L_i1p1f2', 'MIROC6_i1p1f1', 'MPI-ESM1-2-LR_i1p1f1', 'MRI-ESM2-0_i1p1f1', 'MRI-ESM2-0_i2p1f1', 'NESM3_i1p1f1', 'NorESM2-MM_i1p1f1', 'TaiESM1_i1p1f1', 'UKESM1-0-LL_i1p1f2']


/tmp/ipykernel_55437/3393259602.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  event.value = float(Yo.loc[event.time])


In [7]:
n_sample    = 1000
verbose=False

In [8]:
## Prior Functions (Yoann this is so slow)
##================================
t0 = tm.time()
clim = ns.Climatology( event , time_period , models ,n_sample  , ns_law )
Xebm   = ns.EBM().draw_sample( clim.time , n_sample + 1 , fix_first = 0 )
clim =ns.covariates_FC_GAM( clim , lX ,  Xebm , dof = 7 , verbose = False )
clim = ns.nslaw_fit( lY , clim , verbose = verbose ) 
clim = ns.infer_multi_model( clim , verbose = verbose )
## Add constraint on X
clim_light_MM = clim.copy()
clim_light_MM.keep_models( "Multi_Synthesis" )
clim_CX     = ns.constrain_covariate( clim_light_MM , Xo , time_reference , verbose = verbose )
t1 = tm.time()
total = t1-t0
print(total)

KeyboardInterrupt: 

In [ ]:
clim_CX.to_netcdf( "clim_CX_test_"+str(n_sample)+".nc")

In [ ]:
clim_CX=ns.Climatology.from_netcdf(  "clim_CX_test_"+str(n_sample)+".nc" , ns_law )

In [ ]:
#Metropolis Hasting Within Gibbs
t0 = tm.time()
bayes_kwargs = { "n_mcmc_drawn_min" :  5000 , "n_mcmc_drawn_max" : 10000 ,"n_ess"   : 100, "burn_in" : 800  ,"mcmc_init":np.array([0]*5)      }
bayes_kwargs["transition_type"]="Adapt"
bayes_kwargs["method_MCMC"]="bayesian-experimental-mhwg-ess"
climCXCB    = ns.constrain_law( clim_CX , Yo , verbose = verbose , **bayes_kwargs )
t1 = tm.time()
total = t1-t0
print(total)

In [ ]:
#stan version
bayes_kwargs = { "n_ess"   : 100}#int(10000/(len(clim_CX.data.sample)-1))   } #Ici produit 10000 tirages en tout
t0 = tm.time()
climCXCB_stan_light=ns.stan_constrain(clim_CX,Yo,'stan_files/GEV_non_stationary.stan', **bayes_kwargs)
t1 = tm.time()
total = t1-t0
print(total)
climCXCB_stan_light.to_netcdf( ("clim_CXCB_stan.nc")  ) 

In [ ]:
#original with 100 ess
t0 = tm.time()
bayes_kwargs = { "n_mcmc_drawn_min" :  5000 , "n_mcmc_drawn_max" : 10000 ,"n_ess"   : 100, "burn_in" : 800  ,"mcmc_init":np.array([0]*5)      }
bayes_kwargs["transition_type"]="Fixed"
bayes_kwargs["method_MCMC"]="bayesian-experimental-ess"
climCXCB   = ns.constrain_law( clim_CX , Yo , verbose = verbose , **bayes_kwargs )
t1 = tm.time()
total = t1-t0
print(total)

In [ ]:
#Original

In [10]:
## Prior Functions (Yoann this is so slow)
##================================
n_sample    = 1000
t0 = tm.time()
clim = ns.Climatology( event , time_period , models ,n_sample  , ns_law )
Xebm   = ns.EBM().draw_sample( clim.time , n_sample + 1 , fix_first = 0 )
clim =ns.covariates_FC_GAM( clim , lX ,  Xebm , dof = 7 , verbose = False )
clim = ns.nslaw_fit( lY , clim , verbose = verbose ) 
clim = ns.infer_multi_model( clim , verbose = verbose )
## Add constraint on X
clim_light_MM = clim.copy()
clim_light_MM.keep_models( "Multi_Synthesis" )
clim_CX     = ns.constrain_covariate( clim_light_MM , Xo , time_reference , verbose = verbose )
t1 = tm.time()
total = t1-t0
clim_CX.to_netcdf( "clim_CX_test_"+str(n_sample)+".nc")
print(total)

3156.65008020401


In [11]:
clim.to_netcdf( "clim_test_"+str(n_sample)+".nc")

In [12]:
#Original
clim_CX=ns.Climatology.from_netcdf(  "clim_CX_test_"+str(n_sample)+".nc" , ns_law )
t0 = tm.time()
bayes_kwargs = { "n_mcmc_drawn_min" :  5000 , "n_mcmc_drawn_max" : 10000 }
bayes_kwargs["transition_type"]="Fixed"
bayes_kwargs["fixed_cov"]=np.array([0.1,0.1,0.1,0.1,0.1])
climCXCB   = ns.constrain_law( clim_CX , Yo , verbose = verbose , **bayes_kwargs )
t1 = tm.time()
total = t1-t0
print(total)

/home/barbauxo/Documents/05_Autres Activités/2023_11_Transfert_Code_Yoann/SDFC-python/SDFC/__AbstractLaw.py:326: RuntimeWarning: invalid value encountered in scalar subtract
  p_accept = np.exp( p_next - p_current )


2979.580202817917


In [14]:
climCXCB.to_netcdf( "clim_CXCB_test_"+str(n_sample)+".nc")